### Install

In [0]:
%pip install kafka-python

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


### Creating topics

In [0]:
%sh
./kafka_2.12-3.5.1/bin/kafka-topics.sh --bootstrap-server localhost:9092 --create --topic topic_news --partitions 1 --replication-factor 1

# List topics
# ./kafka_2.12-3.5.1/bin/kafka-topics.sh --bootstrap-server localhost:9092 --list

topic_news


### Client

In [0]:
import requests
import json
from kafka import KafkaProducer

class GenomicAPIClient:

    def __init__(self, brokers: list):
        self.brokers = brokers
        self.kafka_producer = KafkaProducer(bootstrap_servers = brokers)
        self.__url = "https://newsapi.org/v2/everything"
        self.__api_key = "efd96606e3804d93ad015c3dd53a764f"

    def __get_headers(self):
        return {
            "Authorization": f"Bearer {self.__api_key}",
            "Content-Type": "application/json"
        }

    def __params(self):
        return {
            'q': 'DNA OR GENÉTICAS OR SEQUENCIAMENTO OR DOENÇAS OR TERAPIAS',
            'language': 'pt',
            'sortBy': 'relevancy'
        }

    def get_data(self):
        response = requests.get(
            url=self.__url,
            headers=self.__get_headers(),
            params=self.__params()
        )

        return response.json() 

    def publish_news(self):
        news_relevancy = self.get_data()

        for news in news_relevancy['articles']:           
            print(f"Send news: {news['source']['name']} to topic_news")
            self.kafka_producer.send("topic_news", json.dumps(news).encode("utf-8"))
            print(news)

        print(f'Send news finish')   

In [0]:
pipeline = GenomicAPIClient(brokers=["localhost:9092"])

pipeline.publish_news()

Send news: Tecmundo.com.br to topic_news
{'source': {'id': None, 'name': 'Tecmundo.com.br'}, 'author': 'Jorge Marin', 'title': 'Google DeepMind: nova ferramenta de IA prevê doenças genéticas', 'description': 'Um novo programa de inteligência artificial desenvolvido por cientistas do Google DeepMind é capaz de prever se milhões de mutações genéticas presentes em nossa sequência de DNA\xa0que são inofensivas ou propensas a causar doenças no futuro. Batizada como AlphaM…', 'url': 'https://www.tecmundo.com.br/ciencia/271822-google-deepmind-nova-ferramenta-ia-preve-doencas-geneticas.htm', 'urlToImage': 'https://tm.ibxk.com.br/2023/09/20/20120248396161.jpg', 'publishedAt': '2023-09-20T15:15:00Z', 'content': 'Um novo programa de inteligência artificial desenvolvido por cientistas do Google DeepMind é capaz de prever se milhões de mutações genéticas presentes em nossa sequência de DNA\xa0que são inofensivas o… [+2940 chars]'}
Send news: Olhardigital.com.br to topic_news
{'source': {'id': None,